# Sherpa on simple sine custom model

Let's first see whether I can make this work on a simple custom model with only two parameters: a sine with an amplitude `arg` and phase `ph`.

Following this example;

https://sherpa.readthedocs.io/en/4.11.0/quick.html

While taking this as template for custom model:

https://sherpa.readthedocs.io/en/4.11.0/model_classes/usermodel.html#usermodel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sherpa.models import model
from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

## Define the custom model

### First the sine function taking all params and an independent variable

In [ ]:
def _make_sine(pars, x):
    """Test function"""
    (arg, ph) = pars
    y = arg * np.sin(x + ph)
    return y

### Now the custom model class

In [ ]:
class SineTest(model.RegriddableModel1D):
    """Test model class"""
    
    def __init__(self, name='sine'):
        self.arg = model.Parameter(name, 'arg', 2, min=0.1, hard_min=0)
        self.ph = model.Parameter(name, 'ph', np.pi)

        model.RegriddableModel1D.__init__(self, name,
                                          (self.arg, self.ph))

    def calc(self, pars, x, *args, **kwargs):
        """Evaluate the model"""

        # If given an integrated data set, use the center of the bin
        if len(args) == 1:
            x = (x + args[0]) / 2

        return _make_sine(pars, x)

## Create test data

And display with `matplotlib`

In [ ]:
np.random.seed(0)
x = np.linspace(-5., 5., 200)
arg_true = 3
ph_true = np.pi + np.pi/3
sigma_true = 0.8
err_true = 1.

y = arg_true * np.sin(x + ph_true)
y += np.random.normal(0., err_true, x.shape)

plt.scatter(x, y, s=3)
plt.title('Fake data')

## Create data object
And display the data with `sherpa`.

In [ ]:
d = Data1D('example_sine', x, y)   # create data object
dplot = DataPlot()         # create data *plot* object
dplot.prepare(d)   # prepare plot
dplot.plot()

## Define the model

In [ ]:
s = SineTest()
print(s)

Visualize the model.

In [ ]:
mplot = ModelPlot()
mplot.prepare(d, s)
mplot.plot()

You can also combine the two plot results to see how good or bad the current model is.

In [ ]:
dplot.plot()
mplot.overplot()

## Select the statistics

Let's do a least-squares statistic, which calculates the numerical difference of the model to the data for each point:

In [ ]:
stat = LeastSq()

## Select optimization

Using Levenberg-Marquardt:

In [ ]:
opt = LevMar()
print(opt)

## Fit ithe data

### Set up the fit

In [ ]:
sfit = Fit(d, s, stat=stat, method=opt)
print(sfit)

### Actually fit the data

In [ ]:
sres = sfit.fit()
print("Fit succeeded?")
print(sres.succeeded)

In [ ]:
# Show fit results
print(sres.format())

The `LevMar` optimiser calculates the covariance matrix at the best-fit location, and the errors from this are reported in the output from the call to the `fit()` method. In this particular case - which uses the `LeastSq` statistic - the error estimates do not have much meaning. As discussed below, Sherpa can make use of error estimates on the data to calculate meaningful parameter errors.

In [ ]:
# Plot the fit over the data
fplot = FitPlot()
mplot.prepare(d, s)
fplot.prepare(dplot, mplot)
fplot.plot()

In [ ]:
# Extracting the parameter values
print(sres)

In [ ]:
ans = dict(zip(sres.parnames, sres.parvals))
print(ans)

In [ ]:
print("The fitted parameter 'arg' is: {:.2f}".format(ans['sine.arg']))

The model, and its parameter values, can alsobe queried directly, as they have been changed by the fit:

In [ ]:
print(s)

In [ ]:
print(s.arg)

## Including errors

In [ ]:
dy = np.ones(x.size) * err_true

# Create data with errors
de = Data1D('sine-w-errors', x, y, staterror=dy)
print(de)

# Plot the data - it will have error bars now
deplot = DataPlot()         # create data *plot* object
deplot.prepare(de)   # prepare plot
deplot.plot()

The statistic is changed from least squares to chi-square (Chi2), to take advantage of this extra knowledge (i.e. the Chi-square statistic includes the error value per bin when calculating the statistic value):

In [ ]:
ustat = Chi2()

In [ ]:
# Do the fit
se = SineTest("sine-err")
sefit = Fit(de, se, stat=ustat, method=opt)
seres = sefit.fit()
print(seres.format())
if not seres.succeeded: print(seres.message)

Since the error value is independent of bin, then the fit results should be the same here (that is, the parameters in `s` are the same as `se`):

In [ ]:
print(s)
print(se)

The difference is that more of the fields in the result structure are populated: in particular the rstat and qval fields, which give the reduced statistic and the probability of obtaining this statistic value respectively.:

In [ ]:
print(seres)

## Errors from Hessian

In [ ]:
calc_errors = np.sqrt(seres.extra_output['covar'].diagonal())

arg_err = calc_errors[0]
ph_err = calc_errors[1]

print('arg_err: {}'.format(arg_err))
print('ph_err: {}'.format(ph_err))

## More thorough error analysis

Proceed as in:

https://sherpa.readthedocs.io/en/4.11.0/quick.html#error-analysis

## More stuff:

On the data class:
https://sherpa.readthedocs.io/en/4.11.0/data/index.html

Model instances - freezing and thawgin parameters, ressetting them, limits, etc.:
https://sherpa.readthedocs.io/en/4.11.0/models/index.html#

Evaluating the model:
https://sherpa.readthedocs.io/en/4.11.0/evaluation/index.html